The aim of the notebook is run inference on the test dataset using the faster-rcnn model trained for 12 epochs. This is just to show the demo of how to use the mmdet framework for test inference. The accuracy obtained from the submission file created is only 0.027. The mmdet framework can be used to easily train and test multiple models with different backbones, so it will be easier to create ensemble of models for final evaluation.


# Training Notebook Link FasterRCNN

https://www.kaggle.com/gauravsingh1/mmdet-pytorch-framework-training-fasterrcnn-base

# Installing MMDET Framework

In [ ]:
import os
import cv2
import random


In [ ]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

In [ ]:
%%time

print("this will take around 10 mins")
# install dependencies: (use cu101 because colab has CUDA 10.1)
# !pip install -U torch==1.7.0+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html

# install mmcv-full thus we could use CUDA operators
!pip install mmcv-full

In [ ]:
!rm -rf mmdetection
!git clone --branch v2.7.0 https://github.com/open-mmlab/mmdetection.git
%cd mmdetection

!pip install -e .

# install Pillow 7.0.0 back in order to avoid bug in colab
!pip install Pillow==7.0.0

# Test Installation

In [ ]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

# Downloading Sample Checkpoints

In [ ]:
!mkdir checkpoints
!wget -c http://download.openmmlab.com/mmdetection/v2.0/mask_rcnn/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco_bbox_mAP-0.408__segm_mAP-0.37_20200504_163245-42aa3d00.pth \
      -O checkpoints/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco_bbox_mAP-0.408__segm_mAP-0.37_20200504_163245-42aa3d00.pth

# Preparing test anno file: Coco Format

In [ ]:
test_anno = "../../input/vinbigdata-1024-image-dataset/vinbigdata/test"

ids = os.listdir(test_anno)

In [ ]:
img_infos = []
for i, _id in enumerate(ids):
    if '.png' in _id:
        img_infos.append({
                    "license": 0,
                    "url": 'null',
                    "file_name": _id,
                    "height": 1024,
                    "width": 1024,
                    "date_captured": 'null',
                    "id": _id
                })

In [ ]:
img_infos[0]

Over here I am just replacing the contents of validation annotation file with the list of test images. The annotation key in the dictionary will be an empty list. Rest will be same for the test ann file.

In [ ]:
import json
val_anno = '../../input/vinbigdata-coco-dataset-with-wbf-3x-downscaled/vinbigdata-coco-dataset-with-wbf-3x-downscaled/val_annotations.json'

with open(val_anno) as f:
    dd = json.load(f)

dd.keys()
dd['annotations']=[]
dd['images']
dd['images'] = img_infos
with open('./test_ann.json', 'w') as outfile:
    json.dump(dd, outfile)

In [ ]:
val_ids = os.listdir('../../input/vinbigdata-coco-dataset-with-wbf-3x-downscaled/vinbigdata-coco-dataset-with-wbf-3x-downscaled/val_images')

In [ ]:
_classes = ("Aortic_enlargement", "Atelectasis", "Calcification", "Cardiomegaly", "Consolidation", "ILD", "Infiltration", "Lung_Opacity", "Nodule/Mass", "Other_lesion", "Pleural_effusion", "Pleural_thickening", "Pneumothorax", "Pulmonary_fibrosis")

MMDET framework gives the option to pass the extra config options as a command line arguments or one can pass the extra config in an dict variable in the function. The function can then merge this dict on the base config.

In [ ]:

_cfg_options = {"dataset_type" : 'CocoDataset',
"classes" : '''("Aortic_enlargement", "Atelectasis", "Calcification", "Cardiomegaly", "Consolidation", "ILD", "Infiltration", "Lung_Opacity", "Nodule/Mass", "Other_lesion", "Pleural_effusion", "Pleural_thickening", "Pneumothorax", "Pulmonary_fibrosis")''',
"data.train.img_prefix" : '../../input/vinbigdata-coco-dataset-with-wbf-3x-downscaled/vinbigdata-coco-dataset-with-wbf-3x-downscaled/',
"data.train.classes" : '''("Aortic_enlargement", "Atelectasis", "Calcification", "Cardiomegaly", "Consolidation", "ILD", "Infiltration", "Lung_Opacity", "Nodule/Mass", "Other_lesion", "Pleural_effusion", "Pleural_thickening", "Pneumothorax", "Pulmonary_fibrosis")''',
"data.train.ann_file" : '../../input/vinbigdata-coco-dataset-with-wbf-3x-downscaled/vinbigdata-coco-dataset-with-wbf-3x-downscaled/train_annotations.json',
"data.train.type":'CocoDataset',
"data.val.img_prefix" : '../../input/vinbigdata-coco-dataset-with-wbf-3x-downscaled/vinbigdata-coco-dataset-with-wbf-3x-downscaled/',
"data.val.classes" : '''("Aortic_enlargement", "Atelectasis", "Calcification", "Cardiomegaly", "Consolidation", "ILD", "Infiltration", "Lung_Opacity", "Nodule/Mass", "Other_lesion", "Pleural_effusion", "Pleural_thickening", "Pneumothorax", "Pulmonary_fibrosis")''',
"data.val.ann_file" : '../../input/vinbigdata-coco-dataset-with-wbf-3x-downscaled/vinbigdata-coco-dataset-with-wbf-3x-downscaled/val_annotations.json',
"data.val.type" : 'CocoDataset',
"data.test.img_prefix" :  '../../input/vinbigdata-1024-image-dataset/vinbigdata/test/',
"data.test.classes" :  '''("Aortic_enlargement", "Atelectasis", "Calcification", "Cardiomegaly", "Consolidation", "ILD", "Infiltration", "Lung_Opacity", "Nodule/Mass", "Other_lesion", "Pleural_effusion", "Pleural_thickening", "Pneumothorax", "Pulmonary_fibrosis")''',
"data.test.ann_file" :  './test_ann.json',
"data.test.type": 'CocoDataset',   
"model.roi_head.bbox_head.num_classes" : '14',
"evaluation.metric" : 'bbox',
"work_dir": "../vinbig_output",
"load_from" : './checkpoints/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco_bbox_mAP-0.408__segm_mAP-0.37_20200504_163245-42aa3d00.pth',
"total_epochs" :'1'
}


cfg_op = ""
for k, v in _cfg_options.items():
    cfg_op+=f"{k}='{v}' "
print(cfg_op)




In [ ]:
_cfg_options

In [ ]:
_cfg_options['model.roi_head.bbox_head.num_classes']=14

# Adding extra config on BaseConfig

In [ ]:
from mmdet.apis import inference_detector, init_detector, show_result_pyplot
# Choose to use a config and initialize the detector
config = 'configs/faster_rcnn/faster_rcnn_r50_caffe_fpn_mstrain_1x_coco.py'
# Setup a checkpoint file to load
checkpoint = '../../input/vinbig-mmdet-fasterrcnn-base12-epoch/vinbig_output/epoch_10.pth'
# initialize the detector
model = init_detector(config, checkpoint, device='cuda:0', cfg_options=_cfg_options)
model.CLASSES = _classes

# Inference On Single Image

Feel free to change the id variable to see more predictions

In [ ]:
_id = random.randint(1,1098)
# Use the detector to do inference
img = f'../../input/vinbigdata-coco-dataset-with-wbf-3x-downscaled/vinbigdata-coco-dataset-with-wbf-3x-downscaled/val_images/{val_ids[_id]}'
result = inference_detector(model, img)
show_result_pyplot(model, img, result, score_thr=0.3)

# Prediction Sample 

The output of the mmdet network is in the following form:

In [ ]:
result

In [ ]:
import os
os.makedirs("../vinbig_output")

# Train the model using the python command

In [ ]:
# !python tools/train.py ./configs/faster_rcnn/faster_rcnn_r50_caffe_fpn_mstrain_1x_coco.py --cfg-options dataset_type='CocoDataset' classes='("Aortic_enlargement", "Atelectasis", "Calcification", "Cardiomegaly", "Consolidation", "ILD", "Infiltration", "Lung_Opacity", "Nodule/Mass", "Other_lesion", "Pleural_effusion", "Pleural_thickening", "Pneumothorax", "Pulmonary_fibrosis")' data.train.img_prefix='../../input/vinbigdata-coco-dataset-with-wbf-3x-downscaled/vinbigdata-coco-dataset-with-wbf-3x-downscaled/' data.train.classes='("Aortic_enlargement", "Atelectasis", "Calcification", "Cardiomegaly", "Consolidation", "ILD", "Infiltration", "Lung_Opacity", "Nodule/Mass", "Other_lesion", "Pleural_effusion", "Pleural_thickening", "Pneumothorax", "Pulmonary_fibrosis")' data.train.ann_file='../../input/vinbigdata-coco-dataset-with-wbf-3x-downscaled/vinbigdata-coco-dataset-with-wbf-3x-downscaled/train_annotations.json' data.train.type='CocoDataset' data.val.img_prefix='../../input/vinbigdata-coco-dataset-with-wbf-3x-downscaled/vinbigdata-coco-dataset-with-wbf-3x-downscaled/' data.val.classes='("Aortic_enlargement", "Atelectasis", "Calcification", "Cardiomegaly", "Consolidation", "ILD", "Infiltration", "Lung_Opacity", "Nodule/Mass", "Other_lesion", "Pleural_effusion", "Pleural_thickening", "Pneumothorax", "Pulmonary_fibrosis")' data.val.ann_file='../../input/vinbigdata-coco-dataset-with-wbf-3x-downscaled/vinbigdata-coco-dataset-with-wbf-3x-downscaled/val_annotations.json' data.val.type='CocoDataset' data.test.img_prefix='../../input/vinbigdata-coco-dataset-with-wbf-3x-downscaled/vinbigdata-coco-dataset-with-wbf-3x-downscaled/' data.test.classes='("Aortic_enlargement", "Atelectasis", "Calcification", "Cardiomegaly", "Consolidation", "ILD", "Infiltration", "Lung_Opacity", "Nodule/Mass", "Other_lesion", "Pleural_effusion", "Pleural_thickening", "Pneumothorax", "Pulmonary_fibrosis")' data.test.ann_file='./test_ann.json' data.test.type='CocoDataset' model.roi_head.bbox_head.num_classes='14' evaluation.metric='bbox' work_dir='../vinbig_output' load_from='./checkpoints/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco_bbox_mAP-0.408__segm_mAP-0.37_20200504_163245-42aa3d00.pth' total_epochs='1'


# Predicting For All the Test Images

In [ ]:
!python tools/test.py ./configs/faster_rcnn/faster_rcnn_r50_caffe_fpn_mstrain_1x_coco.py ../../input/vinbig-mmdet-fasterrcnn-base12-epoch/vinbig_output/epoch_10.pth --cfg-options dataset_type='CocoDataset' classes='("Aortic_enlargement", "Atelectasis", "Calcification", "Cardiomegaly", "Consolidation", "ILD", "Infiltration", "Lung_Opacity", "Nodule/Mass", "Other_lesion", "Pleural_effusion", "Pleural_thickening", "Pneumothorax", "Pulmonary_fibrosis")' data.train.img_prefix='../../input/vinbigdata-coco-dataset-with-wbf-3x-downscaled/vinbigdata-coco-dataset-with-wbf-3x-downscaled/' data.train.classes='("Aortic_enlargement", "Atelectasis", "Calcification", "Cardiomegaly", "Consolidation", "ILD", "Infiltration", "Lung_Opacity", "Nodule/Mass", "Other_lesion", "Pleural_effusion", "Pleural_thickening", "Pneumothorax", "Pulmonary_fibrosis")' data.train.ann_file='../../input/vinbigdata-coco-dataset-with-wbf-3x-downscaled/vinbigdata-coco-dataset-with-wbf-3x-downscaled/train_annotations.json' data.train.type='CocoDataset' data.val.img_prefix='../../input/vinbigdata-coco-dataset-with-wbf-3x-downscaled/vinbigdata-coco-dataset-with-wbf-3x-downscaled/' data.val.classes='("Aortic_enlargement", "Atelectasis", "Calcification", "Cardiomegaly", "Consolidation", "ILD", "Infiltration", "Lung_Opacity", "Nodule/Mass", "Other_lesion", "Pleural_effusion", "Pleural_thickening", "Pneumothorax", "Pulmonary_fibrosis")' data.val.ann_file='../../input/vinbigdata-coco-dataset-with-wbf-3x-downscaled/vinbigdata-coco-dataset-with-wbf-3x-downscaled/val_annotations.json' data.val.type='CocoDataset' data.test.img_prefix='../../input/vinbigdata-1024-image-dataset/vinbigdata/test/' data.test.classes='("Aortic_enlargement", "Atelectasis", "Calcification", "Cardiomegaly", "Consolidation", "ILD", "Infiltration", "Lung_Opacity", "Nodule/Mass", "Other_lesion", "Pleural_effusion", "Pleural_thickening", "Pneumothorax", "Pulmonary_fibrosis")' data.test.ann_file='./test_ann.json' data.test.type='CocoDataset' model.roi_head.bbox_head.num_classes='14' evaluation.metric='bbox' work_dir='../vinbig_output' load_from='./checkpoints/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco_bbox_mAP-0.408__segm_mAP-0.37_20200504_163245-42aa3d00.pth' total_epochs='1' --out results.pkl



# Plotting loss curve

In [ ]:
# !python tools/analyze_logs.py plot_curve ../../input/vinbig-mmdet-fasterrcnn-base12-epoch/vinbig_output/None.log.json --keys lr --legend run1

In [ ]:
os.listdir("./")

# Running Inference on Test Images

In [ ]:
import pickle

with open('./results.pkl', 'rb') as f:
    data = pickle.load(f)

In [ ]:

import json
with open('./test_ann.json', 'rb') as f:
    ann = json.load(f)

In [ ]:
import pandas as pd
test_df = pd.read_csv('../../input/vinbigdata-original-image-dataset/vinbigdata/test.csv')

In [ ]:
file_ids = [file_name.get('file_name').split('.png')[0] for file_name in ann.get('images')]

In [ ]:
!pip install ensemble_boxes

# WBF on Test Predictions

In [ ]:
from ensemble_boxes import *

In [ ]:
ann_with_pred = zip(file_ids, data)
submission_vals = []
for _id, preds in ann_with_pred:
    boxes = []
    scores = []
    labels = []
    width = test_df[test_df.image_id==_id]['width'].iloc[0]
    height = test_df[test_df.image_id==_id]['height'].iloc[0]  
    for i, pred in enumerate(preds):
        if len(pred):
            for p in pred:
                box = p[:4]/1024
                boxes.append(box)
                score = p[4].astype(float)
                scores.append(score)
                labels.append(i)
    boxes, scores, labels = weighted_boxes_fusion([boxes], [scores], [labels], iou_thr=0.4, skip_box_thr=0.4)
    boxes[:, 0] = boxes[:, 0]*height
    boxes[:, 2] = boxes[:, 2]*height
    boxes[:, 1] = boxes[:, 1]*width
    boxes[:, 3] = boxes[:, 3]*width
    
    scaled_boxes = boxes.astype(int)
    labels = labels.astype(int)
    _id_preds = []
    if len(scaled_boxes):
        for i in range(len(scaled_boxes)):
            _id_preds.append(str(labels[i]))
            _id_preds.append(str(scores[i].round(2)))
            _id_preds.append(str(scaled_boxes[i][0]))
            _id_preds.append(str(scaled_boxes[i][1]))
            _id_preds.append(str(scaled_boxes[i][2]))
            _id_preds.append(str(scaled_boxes[i][3]))
        pred_str = " ".join(_id_preds)
    else:
        pred_str = '14 1 0 0 1 1'
    submission_vals.append([_id, pred_str])



# Creating Submission file

In [ ]:
df = pd.DataFrame(submission_vals, columns = ['image_id','PredictionString'])

In [ ]:
df.head()

In [ ]:
len(df)

In [ ]:
df.to_csv("submission.csv",index=False)

If you have reached till here, Thanks for your time!!